# Query 4

This is an example query on the NEXUS database to obtain data from XNAT based on various metadata that have been provided using the Electronic Lab Notebook. 

Query 4 parameters:

- “Device Types” Type intercranial_sensor               <-- Project Level Form
- “Institute”  Peking University                        <-- Project Level Form
- “Stimuli” VisualStims <-- tasks                       <-- Project Level Form
- “Responses” contains MEEG <-- tasks                   <-- Project Level Form

Query 4 output parameters required:
- list of projects, subjects


In [1]:
import pandas as pd
import setup
import nexussdk as nexus
from SPARQLWrapper import SPARQLWrapper, JSON, POST, GET, POSTDIRECTLY, CSV
from questionnaire_query_helper import generate_sparql_compliant_variable_name, generate_question_hash, query_constructor
config = setup.config
import NexusSparqlQuery as qns

# get script arguments
nexus_deployment = config['nexus']['deployment']
org = config['nexus']['org']
project = config['nexus']['project']
token = config['nexus']['token_file']
f = open(token, 'r')
token = f.read().rstrip()

sparqlview_endpoint = setup.sparqlview_endpoint
sparqlview_wrapper = setup.sparqlview_wrapper

nexus = setup.nexus

print("Connected")


Connected


In [2]:
prefix = """
prefix nxv: <https://bluebrain.github.io/nexus/vocabulary/>
PREFIX fhir: <http://hl7.org/fhir/> 
PREFIX kcnietl: <http://www.camh.ca/kcnietl/> 
PREFIX nidm: <http://purl.org/nidash/nidm#> 
PREFIX prov: <http://www.w3.org/ns/prov#> 
PREFIX sio: <http://semanticscience.org/resource/> 
PREFIX dct: <http://purl.org/dc/terms/> 
PREFIX bids: <https://wolfborg.github.io/nidm/bids.html#>
"""

select_list = [
    '?study_id',
    '?subject_id'
]

base_query = """
        ##### Get Study Information
        ?study_uri a fhir:ResearchStudy .
        ?study_uri fhir:ResearchStudy.identifier/fhir:Identifier.value/fhir:value ?study_id .
        
        #### Get Subject Information        
        ?subject_uri fhir:ResearchSubject.study/fhir:link ?study_uri .
        ?subject_uri fhir:ResearchSubject.identifier/fhir:Identifier.value/fhir:value ?subject_id .
"""

project_filter_list = ['Institute', 'Responses', 'Stimuli', 'Type']
for question in project_filter_list:
    sparql_compliant_variable_name = generate_sparql_compliant_variable_name(question)
    select_list.append(sparql_compliant_variable_name)
    base_query = base_query + query_constructor('?study_uri', question, sparql_compliant_variable_name)

In [3]:
query = f"""
{prefix}

select distinct
    {' '.join(select_list)}
where
    {{
        {base_query}
    }}
"""

df = qns.sparql2dataframe(qns.query_sparql(query, sparqlview_wrapper))

df

,study_id,subject_id,Institute,Responses,Stimuli,Type
0,nexus3,NC001,Donders Institute (Centre for Cognitive Neuroi...,"FMRI, ET, ButtonPress",VisualStims,screen
1,nexus3,NC001,Donders Institute (Centre for Cognitive Neuroi...,"FMRI, ET, ButtonPress",VisualStims,amplifier
2,nexus3,NC001,Donders Institute (Centre for Cognitive Neuroi...,"FMRI, ET, ButtonPress",VisualStims,sensor
3,nexus1,NA002,New York University Langone Health,"ET, ECOG, ButtonPress",VisualStims,intercranial_sensor
4,nexus1,NA001,New York University Langone Health,"ET, ECOG, ButtonPress",VisualStims,intercranial_sensor
5,nexus1,NA002,University of Birmingham,"ET, ECOG, ButtonPress",VisualStims,intercranial_sensor
6,nexus1,NA001,University of Birmingham,"ET, ECOG, ButtonPress",VisualStims,intercranial_sensor
7,nexus1,NA002,New York University Langone Health,"MEEG, ET, ButtonPress",VisualStims,intercranial_sensor
8,nexus1,NA001,New York University Langone Health,"MEEG, ET, ButtonPress",VisualStims,intercranial_sensor
9,nexus1,NA002,University of Birmingham,"MEEG, ET, ButtonPress",VisualStims,intercranial_sensor


In [4]:
df.loc[
    (df['Type'] == 'intercranial_sensor')
    & (df['Responses'].str.find('ButtonPress')!=1)
    & (df['Stimuli'] == 'VisualStims' )
    ].drop_duplicates()

,study_id,subject_id,Institute,Responses,Stimuli,Type
3,nexus1,NA002,New York University Langone Health,"ET, ECOG, ButtonPress",VisualStims,intercranial_sensor
4,nexus1,NA001,New York University Langone Health,"ET, ECOG, ButtonPress",VisualStims,intercranial_sensor
5,nexus1,NA002,University of Birmingham,"ET, ECOG, ButtonPress",VisualStims,intercranial_sensor
6,nexus1,NA001,University of Birmingham,"ET, ECOG, ButtonPress",VisualStims,intercranial_sensor
7,nexus1,NA002,New York University Langone Health,"MEEG, ET, ButtonPress",VisualStims,intercranial_sensor
8,nexus1,NA001,New York University Langone Health,"MEEG, ET, ButtonPress",VisualStims,intercranial_sensor
9,nexus1,NA002,University of Birmingham,"MEEG, ET, ButtonPress",VisualStims,intercranial_sensor
10,nexus1,NA001,University of Birmingham,"MEEG, ET, ButtonPress",VisualStims,intercranial_sensor
11,nexus2,NB002,New York University Langone Health,"ET, ECOG, ButtonPress",VisualStims,intercranial_sensor
12,nexus2,NB001,New York University Langone Health,"ET, ECOG, ButtonPress",VisualStims,intercranial_sensor
